# Kaggle: Predict Loan Payback — Model Training

**Notebook:** `04_model_training.ipynb`
**Author:** Brice Nelson
**Organization:** Kaggle Series | Brice Machine Learning Projects
**Date Created:** November 16, 2025
**Last Updated:** November 19, 2025

---

## 🧭 Purpose

This notebook initiates the **modeling phase** for the *Predict Loan Payback* competition.

After completing data cleaning and feature engineering in previous notebooks, we now transition into selecting, training, evaluating, and comparing machine-learning models capable of predicting whether a borrower will repay the loan.

This step turns the carefully prepared dataset into an **actionable predictive system**.

### **Objectives**
1. Load feature-engineered train/test datasets from `/data/processed/`.
2. Define the target variable and feature matrix.
3. Train baseline models to establish initial performance benchmarks.
4. Evaluate models using appropriate metrics (AUC, accuracy, precision/recall, etc.).
5. Compare multiple algorithms and select the strongest candidate(s).
6. Export predictions for Kaggle submission.

---

## 🧱 Model Training Roadmap

The modeling plan for this notebook includes:

### **1. Baseline Models**
- Logistic Regression (regularized)
- Decision Tree (simple depth-limited version)

Purpose: establish “floor” performance quickly.

---

### **2. Core Machine Learning Models**
- Random Forest
- Gradient Boosting (e.g., XGBoost or LightGBM)
- Extra Trees Classifier
- Support Vector Machine (if practical)

These will form the backbone of your model comparison phase.

---

### **3. Hyperparameter Tuning**
- RandomizedSearchCV for broad sweeps
- GridSearchCV for refining top models
- Evaluation via stratified cross-validation
- Tracking overfitting by comparing train vs. validation scores

---

### **4. Model Evaluation Metrics**
Depending on competition scoring:

- **ROC AUC** (typical for binary classification)
- **Accuracy**
- **Precision / Recall**
- **Confusion matrix**
- **Calibration curves** (optional but useful for loan risk)

---

### **5. Prediction & Export**
- Predict on the processed test dataset
- Format output to match Kaggle’s expected submission CSV
- Save to `/data/submissions/`

---

## 📥 Load Feature-Engineered Data

This notebook begins by importing:

- `../data/processed/loan_train_features.csv`
- `../data/processed/loan_test_features.csv`

(or whichever filenames you created in the feature engineering notebook)

These will be used to construct the feature matrix `X` and target vector `y` for training and validation.


In [11]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report


## Load Processed Data

In [2]:
loan_train_features = pd.read_csv("../data/processed/loan_train_features.csv")
loan_test_features = pd.read_csv("../data/processed/loan_test_features.csv")

---

## ⚙️ Step 1: Define Features and Target

With the feature-engineered datasets loaded, the next step is to construct:

- **X_train** → Feature matrix
- **y_train** → Target vector (`loan_payed_back`)
- **X_test** → Feature matrix for Kaggle submission

This section will:
1. Separate predictors from the target column.
2. Confirm dataset shapes and check for any remaining inconsistencies.
3. Prepare the data for model training and baseline evaluation.

---

## 🧪 Step 2: Baseline Models

Before diving into advanced algorithms, we start with simple baseline models to:

- Establish a performance benchmark
- Verify that our preprocessing is correct
- Catch issues like data leakage or extreme imbalance early

The baseline models we will train:

### **1. Logistic Regression (Regularized)**
A reliable, interpretable starting point for binary classification.

### **2. Decision Tree (Depth-Limited)**
Helps visualize splitting patterns and provides an early non-linear alternative.

We’ll evaluate each using:

- ROC-AUC
- Accuracy
- Precision / Recall
- Confusion matrix

This gives us a solid “floor” before moving into more powerful ensemble methods.

---


In [5]:
# ----------------
# Loan Features Head()
# ----------------
print(f'Loan Train Feature:\n{loan_train_features.head()}')
print(f'Loan Test Featurs: \n{loan_test_features.head()}')

Loan Train Feature:
   id  annual_income  debt_to_income_ratio  credit_score  loan_amount  \
0   0      -0.705461             -0.535135      0.993849    -1.803484   
1   1      -0.977248              0.660668     -0.810394    -1.505401   
2   2       0.050689             -0.345556      0.236067     0.286558   
3   3      -0.050687             -0.812211     -2.668764    -1.492497   
4   4      -0.850388             -0.987206     -0.287163    -0.409421   

   interest_rate  loan_paid_back     grade  subgrade  gender_Female  ...  \
0       0.653899             1.0 -0.401966  0.008691            1.0  ...   
1       0.280571             0.0  0.613154  0.008691            0.0  ...   
2      -1.292385             1.0 -0.401966  1.434819            0.0  ...   
3       1.863482             1.0  2.643393 -1.417436            1.0  ...   
4      -1.068388             1.0  0.613154 -1.417436            0.0  ...   

   grade_x_loan_purpose_Car  grade_x_loan_purpose_Debt consolidation  \
0           

In [7]:
# ----------------
# Loan Features Info()
# ----------------

print('Loan Train Features:\n', loan_train_features.info())
print('Loan Test Features: \n', loan_test_features.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593994 entries, 0 to 593993
Data columns (total 53 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   id                                       593994 non-null  int64  
 1   annual_income                            593994 non-null  float64
 2   debt_to_income_ratio                     593994 non-null  float64
 3   credit_score                             593994 non-null  float64
 4   loan_amount                              593994 non-null  float64
 5   interest_rate                            593994 non-null  float64
 6   loan_paid_back                           593994 non-null  float64
 7   grade                                    593994 non-null  float64
 8   subgrade                                 593994 non-null  float64
 9   gender_Female                            593994 non-null  float64
 10  gender_Male                     

In [8]:
# -----------------------------------------------
# Step 1: Define Features (X) and Target (y)
# -----------------------------------------------

# The target column from the training set
target_col = "loan_paid_back"

# Feature matrix and target for training
X_train = loan_train_features.drop(columns=[target_col])
y_train = loan_train_features[target_col]

# Test set has no target column — that's correct
X_test = loan_test_features.copy()

print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)



X_train: (593994, 52)
y_train: (593994,)
X_test: (254569, 52)


In [10]:
# -----------------------------------------------
# Step 2: Train/Validate Split
# -----------------------------------------------

X_train_split, X_valid, y_train_split, y_valid = train_test_split(
    X_train,
    y_train,
    test_size=0.20,
    random_state=42,
    stratify=y_train
)

print("Train split:", X_train_split.shape)
print("Valid split:", X_valid.shape)


Train split: (475195, 52)
Valid split: (118799, 52)


In [12]:
# -----------------------------------------------
# Step 3: Baseline Logistics Regression
# -----------------------------------------------

log_reg = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    n_jobs=-1
)

log_reg.fit(X_train_split, y_train_split)

# Predictions
y_pred_lr = log_reg.predict(X_valid)
y_prob_lr = log_reg.predict_proba(X_valid)[:, 1]

roc_lr = roc_auc_score(y_valid, y_prob_lr)

print(f"ROC-AUC (Logistic Regression): {roc_lr:.4f}")
print("\nClassification Report:")
print(classification_report(y_valid, y_pred_lr))


/home/brice-nelson/miniconda3/envs/predict_loan_payback/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ROC-AUC (Logistic Regression): 0.9058

Classification Report:
              precision    recall  f1-score   support

         0.0       0.60      0.78      0.68     23900
         1.0       0.94      0.87      0.90     94899

    accuracy                           0.85    118799
   macro avg       0.77      0.82      0.79    118799
weighted avg       0.87      0.85      0.86    118799

